#### 목표 설정
- kiwipiepy 형태소 분석기를 사용해보고 Konlpy와는 어떤점이 다른지 확인

#### Kiwi 라이브러리
- 빠른속도와 범용적인 성능을 지향하는 형태소 분석기
- C++로 구현된 코어를 래핑해서 다양한 프로그래밍 언어에서 사용하도록 만들었다. / 오픈소스 공개
- 세종 품사 태그셋을 사용한다.
- 웹 텍스트 약 87% / 문어체 텍스튼느 94% 정확도로 형태소 분석이 가능하다. ( 공식 GitHub 참조 )
- Konlpy보다 속도가 더 빠르다.
- 간단한 오타나 띄어쓰기는 스스로 교정 할 수 있는 기능을 제공, 신조어나 오타에 강

In [1]:
# 경로 확인
import os
print("현재 작업 경로:", os.getcwd())

# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 불러오기 - WSL 환경에 맞게 경로 수정
# 데이터 파일은 ~/projects/data 폴더에 있다고 가정
train = pd.read_csv('../data/ratings_train.csv', encoding='utf-8')
test = pd.read_csv('../data/ratings_test.csv', encoding='utf-8')

# 데이터 확인
print("학습 데이터 미리보기:")
print(train.head())

print("\n테스트 데이터 미리보기:")
print(test.head())

현재 작업 경로: /home/jsock/projects
학습 데이터 미리보기:
         id                                           document  label
0   9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1   3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2  10265843                                  너무재밓었다그래서보는것을추천한다      0
3   9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1

테스트 데이터 미리보기:
        id                                           document  label
0  6270596                                                굳 ㅋ      1
1  9274899                               GDNTOPCLASSINTHECLUB      0
2  8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0
3  6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
4  6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0


In [3]:
X_train = train['document']
X_test = test['document']
y_train = train['label']
y_test = test['label']

In [5]:
# 데이터 shape 확인
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((150000,), (50000,), (150000,), (50000,))

Kiwipiepy 라이브러리 설치
- Kiwipiepy 라이브러리는 의존성이 없음. 편리하게 환경 구축이 가능하다.

In [6]:
pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 55.8 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 65.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 10.9 MB/s eta 0:00:00
  Created wheel for kiwipiepy_model: filename=kiwipiepy_model-0.20.0-py3-none-any.whl size=34818072 sha256=40f4b10e94f56ce4e5971815aeeef5f78028d791857b355c49e879d62290d1b4
  Stored in directory: /home/jsock/.cache/pip/wheels/f3/14/24/bd3e0c3aa06df12afcb35447b07840213644855333f33bfce5
Successfully built kiwipiepy_model
Note: you may need to restart the kernel to use updated packages.


In [3]:
# 라이브러리 import
# 사이킷런 Tfidf Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Kiwipiepy 형태소 분석기 import
from kiwipiepy import Kiwi, basic_typos_with_continual

In [5]:
# 객체 생성
kiwi = Kiwi(typos = basic_typos_with_continual)
kiwi

Kiwi(num_workers=1, model_path=None, integrate_allomorph=True, load_default_dict=True, load_typo_dict=True, model_type='knlm', typos=TypoTransformer([('\x00가', '까', 1.0, 'applosive'), ('\x00개', '깨', 1.0, 'applosive'), ('\x00갸', '꺄', 1.0, 'applosive'), ('\x00걔', '꺠', 1.0, 'applosive'), ('\x00거', '꺼', 1.0, 'applosive'), ... (11241 more)], continual_typo_cost=1.0, lengthening_typo_cost=inf), typo_cost_threshold=2.5)

- Kiwipiepy는 간단한 오타 교정도구를 제공
  - basic_typos : 형태소 내에 오타를 교정하는 기본적인 오타 교정 도구
  - continual_typos : 형태소간의 연철 오타를 교정하는 교정 도구
  - basic_typos_with_continual : 위의 두가지 오타 교정도구를 하나로 합친 기능

In [6]:
# 형태소 분석 - tokenize
token = kiwi.tokenize('아버지가방에들어가신다')
token

# form : 실제 형태소
# tag : 품사 태그
# start : 형태소의 시작 인덱스
# len : 형태소의 길

[Token(form='아버지', tag='NNG', start=0, len=3),
 Token(form='가', tag='JKS', start=3, len=1),
 Token(form='방', tag='NNG', start=4, len=1),
 Token(form='에', tag='JKB', start=5, len=1),
 Token(form='들어가', tag='VV', start=6, len=3),
 Token(form='시', tag='EP', start=9, len=1),
 Token(form='ᆫ다', tag='EF', start=9, len=2)]